In [1]:
from skimage.io import imread, imsave

import surface_extraction

In [29]:
#  Read image stack
img_path = "data//"
img_file = "human_tongue_tissue.tif"
img_stack = imread(img_path + img_file).astype('uint16')

###  Surface extraction

In [30]:
#  Parameter settings
surf = surface_extraction.SurfaceExtraction(img_stack)
surf.cluster_method = 'Kmeans'  # Pixel classification - FastSME
lambda1, edge_flag2, ind_max =  surf.parameter_setting()

Initialization...
FFT on z profiles...
Clustering z profiles...
Initialization time: 5.62s
Parameter settings...
Parameter settings time: 0.34s


In [31]:
#  Smooth manifold extraction
surf.WW = 1.8  # Adjust local smoothness
surf.THRES = 0.1  # Stop criteria
ind_maxk = surf.surface_smooth(lambda1, edge_flag2, ind_max)

Smoothing surface...
Surface smoothing time: 9.25s


###  Rapid surface extraction (GPU parallel computing)

mSME extracts a more comprehensive surface, especially on large surfaces with various signal types.

In [35]:
#  Parameter settings
r_surf = surface_extraction.RapidSurfaceExtraction(img_stack)
r_surf.cluster_method = 'Projection'  # Pixel classification - mSME
r_surf.RLX = 0.5  # Improve low-intensity signal extraction
lambda1, edge_flag2, ind_max =  r_surf.rapid_parameter_setting()

Initialization...
FFT on z profiles...
Thresholding z profiles...
Initialization time: 5.35s
Parameter settings...
Parameter settings time: 0.30s


GPU acceleration is more significant on large image stacks where the processing time can be prohibitive.

In [36]:
#  Smooth manifold extraction
r_surf.WW = 1.8  # Adjust local smoothness
r_surf.THRES = 0.1  # Stop criteria
ind_maxk = r_surf.surface_smooth(lambda1, edge_flag2, ind_max)

Smoothing surface...
Resolution scale 0
Allocated device memory: 47.2MB
Memory pool usage: 55.1MB
Resolution scale 1
Allocated device memory: 89.6MB
Memory pool usage: 107.9MB
Resolution scale 2
Allocated device memory: 259.2MB
Memory pool usage: 272.2MB
Surface smoothing time: 2.49s


### Saving results

Cutting artifacts are seen on the right side of the superficial surface

In [38]:
#  Extract superficial surface
d = 0
ind_surf, surface = r_surf.surface_extract(ind_maxk, depth=d, continuity=True)
#  Save surface
output_path = img_path + "results//"
imsave(output_path + "surf_d" + str(d) + "_" + img_file, surface.astype('uint16'))

Extracting surface at depth 0


A better surface with clear tissue structures (~10 $\mu m$ beneath actual surface)

In [39]:
#  Extract surface at depth 5
d = 5
ind_surf, surface = r_surf.surface_extract(ind_maxk, depth=d, continuity=True)
#  Save surface
output_path = img_path + "results//"
imsave(output_path + "surf_d "+ str(d) + "_" + img_file, surface.astype('uint16'))

Extracting surface at depth 5
